# ARCDAP-3 ESMValTool Worksheet 

---
## Module 1  

---

In [ ]:
# created by Gerald on 17 Jan 2020

# copy & paste the ESM_Worksheets folder into the ~/ESMValtool/ directory and open it there.

Welcome to Module 1 of the ESMValTool Worksheet designed for the hands-on sessions during the ARCDAP-3 workshop. Module 1 will cover the aims and instructions for this worksheet as well the Tasks 0 and 1 of the hands-on sessions. Please complete the worksheet modules in order from 1 to 3. 

Enter your details in the cell below:

In [ ]:
# Name: 
# Organisation: 
# GCMs used: 

In [ ]:
#imports 

import numpy as np
import scipy as sp
import xarray as xr

from IPython.display import Image, display

import fnmatch # find match

import glob
from pprint import pprint
from pathlib import Path

### Aims

This worksheet is designed as a starting point for ARCDAP-3 participants to use ESMValTool and Python (through Jupyter Notebook) to perform routine model evaluation diagnostics on global climate models (GCMs) from the CMIP database. Some familiarity with the Python and Unix language, along with netCDF files is required to perform the exercises in this worksheet. By the end of ARCDAP-3, you should have successfully evaluated for your assigned GCM(s): <br>
* Plots of (and) mean 850 hPa air temperature and precipitation flux from 2000 - 2002. **(Module 1)** 
* Plots of (and) mean surface temperature and precipitation flux biases from 1995 - 2014. **(Module 2)** 
* Plots of trends in sea-surface temperatures, ENSO and IOD indices from 1979-2014. **(Module 3)**  
* Short write-ups on your results and observations  

### Instructions

You may complete this worksheet individually or as a pair; CMIP GCMs have been assigned to participating countries in pairs. Follow the instructions to write your own code or make edits to the code cells wherever prompted by the comments (denoted by #).

For the GCMs that you’ll work on, refer to the model assignment and download (or copy from one of the available hard drives) the model data assigned to you e.g. if you’ve been assigned the CNRM-CM6-1 model, download all 4 variable files (tas, ts, psl, pr) from this model. **Check that you have downloaded the correct files before proceeding.** Note the slightly different file name conventions between CMIP generations: <br>

CMIP5: 
* ```[variable_short_name]_[mip]_[dataset_name]_[experiment]_[ensemble]_[start-date]-[end-date].nc```

CMIP6:
* ```[variable_short_name]_[mip]_[dataset_name]_[experiment]_[ensemble]_[grid]_[start-date]-[end-date].nc```

Also download the *ERA-Interim* folder containing ERA-Interim data for the same 4 variables and *HadISST* folder containing the *HadISST_sst.nc* file. You should have thus downloaded a total of **13 data files**. 

### Some Orientation
You'll be using the Jupyter Notebook cells to run ESMValTool and other Unix commands as if you are using a terminal. If you have followed the instructions correctly, you should have opened this Notebook in the *~/ESMValTool* directory (*~* is just shortcut for your home directory which will be */home/arcdap* if you're using the VM) with the esmvaltool environment activated. This notebook should be running on a Python 3 kernel, meaning that scripts you write into the code cells are interpreted as Python. 

In [ ]:
# Test out this Python script by pressing Shift+Enter while the cell 
# is in edit mode 
A = 1
A < 2

However you can also use the Jupyter cells to call Unix commands as though you are in a terminal by simply adding ```!``` at the front of your code.  

In [ ]:
# Test out this Shell script by pressing Shift+Enter while the cell 
# is in edit mode

!ls #ls command lists all the files in your current directory, ~/ESMValTool 

## Task 0: Prerequisites 
---

**Data and Scripts needed:** <br>
Model and Observation output: <br>
- 2 x CMIP6 Models + 1 x ERA-Interim <br>
- CMIP experiment: historical <br>
- Variables: tas, ts, pr, psl  <br>
- Frequency: Monthly mean <br>

Raw Observation output: <br>
- 1 x HadISST <br>
- Variables: ts/SST <br>
- Frequency: Monthly mean <br>
    
---

Before running ESMValTool, you have to first organise your downloaded CMIP6, ERA-Interim and HadISST files and properly set up the configuration file that ESMValTool reads to know where to find your data. For standardisation, please follow the steps below to ensure that everyone will have the same set-up of how and where their data are stored to make any debugging later easier. 

#### Organising your data

1. Open up your home folder by clicking on MENU and selecting "File Manager"(for users using the VM). In your */home/arcdap* folder you should see that a folder *Models* has already been created for you. Inside this folder, you should find “ta” and “pr” netCDF (.nc) files belonging to the MPI-ESM-LR from CMIP5 that have been pre-downloaded. **Move or Copy** only the CMIP6 .nc files that you downloaded to this *Models* folder. You will use this folder to store all the CMIP6 .nc files you have downloaded (without any further subfolders). 

<img src='Images/models_folder.jpg' width="400">

<h5 align="center">All CMIP output .nc files in the same ~/Models directory.<h5>

2. For the ERA-Interim observation "OBS6" files, first create a directory in */home/arcdap* called *Obs*. Inside *Obs*, create a sub-folder called *Tier3*  then copy/move the *ERA-Interim* folder that you downloaded into this *Tier3* folder. (*This tier corresponds to  data tiers where Tier3 refers to restricted datasets i.e. requires a registration to be retrieved or provided upon request to the respective contact or PI*). Your ERA-Interim .nc data files should now be inside the */home/arcdap/Obs/Tier3/ERA-Interim* directory. 

3. For the HadISST raw observation file *HadISST_sst.nc*, first create a directory in */home/arcdap* called *RawObs*. Inside *RawObs*, create a sub-folder called *Tier2* and then copy/move the *HadISST* folder that you downloaded into this *Tier2* folder. Your *HadISST_sst.nc* file should now be inside the */home/arcdap/RawObs/Tier2/HadISST* directory. 

#### Setting up the ESMValTool configuration file. 

You will now set up the configuration file in ESMValTool to direct it to find CMIP and Observation files in the *~/Models*, *~/Obs* and *~/RawObs*  directories. 
1. Open up the *config-user-example.yml* file with the cell below.

In [ ]:
# geany is the name of the text editor that you're commanding to open to edit the config file
!geany ~/ESMValTool/config-user-example.yml

2. In the *config-user-example.yml* file, set the paths for the ```CMIP5``` and ```CMIP6``` directories under ```rootpath``` to the directory ```~/Models```. Create a new line called ```OBS6``` and set the path for ```OBS6``` and ```OBS``` to ```~/Obs```. Lastly, set the path for ```RAWOBS``` to ```~/RawObs```. Your configuration file should resemble the image below. 

<img src='Images/config.PNG'>

3. Scroll to the bottom and ensure that the options ```write_plots``` and ```write_netcdf``` are both set to ```true```. Leave all other settings as their defaults. Save and close your *config-user-example.yml* file.

Well done! You're now all set to start the worksheet proper. 

## Task 1: Getting Started

--- 
**Data and Scripts needed:** <br>
Model output: <br>
- 1x CMIP5 Model output (MPI-ESM-LR) <br>
- CMIP experiment: historical <br>
- Variables: ta, pr <br>
- Frequency: Monthly mean <br>

Recipe: *~/ESMValTool/esmvaltool/recipes/examples/recipe_python.yml*

---

### T1.1. Create plots of mean air temperature and precipitation flux from 2000 to 2002 for one of your assigned GCMs

This starting task will help you get used to the workflow and design of ESMValTool. You will use a simple example recipe to plot the model’s mean 850 hPA air temperature (ta) and precipitation flux (pr) over a short time period. 

Using the code cell below, navigate to the esmvaltool/recipes/examples/ directory and open the recipe_python.yml file. 

In [ ]:
!geany esmvaltool/recipes/examples/recipe_python.yml

The first section of the recipe documents metadata of the recipe such as the recipe’s description and authors. The second section includes the declaration of datasets to be evaluated with this recipe. You should see something like this: 

<img src='Images/recipe_python.PNG'>

You should understand the first line under ```datasets``` (enclosed in { }) as referring to the CanESM2 model, from the CMIP5 project. The frequency of data to be used is monthly (“Amon” refers to Atmospheric Monthly), the data from the CMIP “historical” experiment and from the “r1i1p1” ensemble member. ESMValTool identifies the model output to be processed based on how these are declared and whether they match with the names of each model's respective .nc files. 


The recipe will then be performed over data from the year 2000 to 2002. 

You'll need to edit the ```datasets``` section so that the recipe only runs for the models that you want to run the recipe on. You only have output from the CMIP5 MPI-ESM-LR model, hence keep the line which belongs to this model and remove/comment out the unwanted models e.g. CanESM2. <br>

---
*You can also comment out lines in yml using “#” at the start, similar to Python. Yml however not does support multi-line commenting.*

---

Moving down, the preprocessors section indicates that before the actual recipe is run, the GCM air temperature data is first extracted from the 85000 Pa level and that a linear re-gridding scheme is used. Lastly, the diagnostics section provides a short description of what the actual diagnostic code, *examples/diagnostic.py* does, as well as the variables required, “ta” and “pr”. Save and close the recipe file. 

You are now ready to run your first ESMValTool recipe! 

1. Running recipes in ESMValTool typically follow a structure of:

               esmvaltool -c [Configuration File] [Recipe File]

2. Run the recipe in the cell below: 

If your recipe runs successfully, you will see a ```Run was successful``` message at the end of the output. _Please feel free to consult any of the experts should you encounter an error or bug. Only proceed if your recipe runs successfully._

In [ ]:
# Type in the esmvaltool recipe to run after the config-user-example
# file. What is the path of the recipe file your need (see "Data and 
# Scipts needed"). What is your present directory? 
! esmvaltool -c config-user-example.yml 

3. Now, navigate to the directory containing the output from running this recipe. By default, all recipe output files are stored inside the *~/ESMValTool/esmvaltool_output* directory.

In [ ]:
! ls esmvaltool_output/

4. The output for your recipe will be stored in a sub-directory structured as *[recipe name] _ [YYYYMMDD] _ [HHMMSS]*. The plots of mean “ta” and “pr” should be in the */plots* folder within this directory. Use the code below to open and display the plots. 

In [ ]:
home = str(Path.home())  #Your home directory

# Insert the sub-folder containing the recipe output into the ''
# e.g. dirname1='recipe_python_20200117_070100'
dirname1='';  
# Full path of the folder containing the plots produced by the recipe 
dirname=home+'/ESMValTool/esmvaltool_output/'+ dirname1 +'/plots/diagnostic1/script1/';

# List of .png image files in the dirname directory
listing = glob.glob(dirname+'*.png')

pprint(listing)

In [ ]:
# Display the plots of mean ta and pr:

for imageName in listing:
    display(Image(filename=imageName))
    

### T1.2. Calculate the mean air temperature and precipitation flux from 2000 to 2002 for your assigned GCM

Congratulations! You’ve successfully run and obtained output from your first ESMValTool recipe. However, suppose you want to calculate the global mean air temperature and precipitation flux from these plots, how can you do so? 

1. Because you've set the option for ```write_netcdf``` to ```true``` in the *config-user-example.yml* file in the prerequesites task, ESMValTool has saved the data that generates the above two plots in the .nc format. These will be located in this recipe's output directory in the */work* sub-folder.

In [ ]:
dirname2=home+'/ESMValTool/esmvaltool_output/'+ dirname1 +'/work/diagnostic1/script1/';

# List of .png image files in the dirname directory
# Following the example from above for plots, how will you extract the list 
# of .nc files in the directory? Insert your answer in the quotations ''. 
listing2 = glob.glob(dirname2+'')

pprint(listing2)

2. Use the xarray library to open the first .nc file for mean precipitation flux (pr) from 2000 to 2002. 

In [ ]:
# Open the .nc file containing the pr variable 
# Recall that you can open the file from the 'listing2' list 
# The pr.nc file is the 0th index, i.e. listing2[0]
MPI_1 = xr.open_dataset(listing2[0])
MPI_1

3. Extract the mean pr data from the ```'pr'``` key from ```"Data variables"```.

In [ ]:
MPI_pr = MPI_1['pr']
MPI_pr

4. Use Numpy to calculate the global mean pr for the MPI-ESM-LR model. <br>

**Global mean precipitation flux (pr):**

To calculate the global mean we must also factor in the area of every grid, which varies according to location (since Earth is a sphere and not a flat projection). We'll use for this hands-on exercise, a simple *cosine-weighting* technique to factor in the convergence of the meridians at higher latitudes (i.e. reduce the weights of grids at high latitudes which represent a smaller globe area). 

Cosine weighting:

In [ ]:
# First we'll need the latitude data of the output file. 
MPI_1['lat'].data

In [ ]:
# Take the zonal average first (axis=1), then weigh each latitude by cosine(latitude),
# then take the weighted average 
# Remember that the latitude angles must be convertedn into radians

np.average(np.mean(MPI_pr.data, axis=1), weights = np.cos(np.pi*MPI_1['lat'].data/180))

5. Repeat the above for 850 hPa air temperature from 2000 to 2002. 

In [ ]:
# Fill in the appropriate argument into xr.open_dataset()
MPI_2 = xr.open_dataset()
# Fill in the appropriate key into MPI_2 to extract air temperature (ta)
MPI_ta = MPI_2[]
MPI_ta

You should notice that there are some extremely large (e+36) values in the array. These artefacts should be masked before you perform any calculations on the data array as they can interfere with your calculations. A simple method to do so is shown below:

In [ ]:
MPI_ta = MPI_ta.where(MPI_ta < 10**5)

#You will see that those values have been converted to nan
MPI_ta

Now compute the following, 

**Global mean 850 hpa air temperature (ta):**

In [ ]:
MPI_ta.data

In [ ]:
# Same as before, unfortunately np.nanaverage does not exist, but nansum does, 
# so we'll just do the weighted average calculation in parts to ignore the 
# nan values. 

weights = np.cos() #Complete this line 
weighted_sum = np.nansum(np.nanmean(MPI_ta.data, axis=1)*weights)

# You already know that the 1st and last rows are nan, so ignore those rows 
# when summing the weights with [1:-1]
ta_avg = weighted_sum/np.sum(weights[1:-1])
ta_avg

---
**Congratulations!** You’ve completed the starting tutorials for ESMValTool. You’ll now move on to computing more in-depth model evaluation with ESMValTool and Python.The subsequent exercises will be more difficult with less “hand-holding” involved. Nevertheless, feel free to approach our ESMValTool developers and experts if you’re stuck or facing any issues. Happy coding!


---

In [ ]:
# end of file, Gerald, last edited 21/1/2020.